# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning I</font>

# Salvando e Carregando Modelos Treinados

## Definindo um Grafo Multilayer Perceptron 

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# Obs: Este script está compatível com as versões 1.x e 2.x do TensorFlow.
# Optamos por manter assim, pois alguns recursos avançados usados neste script ainda não foram implementados no TF 2.

# Para executar este script com TF 2, nenhum passo adicional precisa ser feito.
# Para executar com TF 1, remova o prefixo tf.compat.v1 ao longo do scriipt e substitua por tf, e comente as linhas abaixo.
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.compat.v1.disable_eager_execution()

In [ ]:
import tensorflow as tf
from warnings import filterwarnings
filterwarnings('ignore')


##########################
### Definindo o Modelo
##########################

# Função que define uma camada
def fc_layer(input_tensor,
             n_output_units,
             name,
             activation_fn = None,
             seed = None,
             weight_params = None,
             bias_params = None):

    with tf.compat.v1.variable_scope(name):

        if weight_params is not None:
            weights = tf.constant(weight_params, name = 'weights', dtype = tf.float32)
        else:
            weights = tf.Variable(tf.random.truncated_normal(
                shape=[input_tensor.get_shape().as_list()[-1], n_output_units],
                    mean = 0.0,
                    stddev = 0.1,
                    dtype = tf.float32,
                    seed = seed),
                name = 'weights',)

        if bias_params is not None:
            biases = tf.constant(bias_params, name = 'biases', dtype=tf.float32)

        else:
            biases = tf.Variable(tf.zeros(shape = [n_output_units]),
                                 name = 'biases',
                                 dtype = tf.float32)

        act = tf.matmul(input_tensor, weights) + biases

        if activation_fn is not None:
            act = activation_fn(act)

    return act

# Função que define o grafo
def mlp_graph(n_input = 784,
              n_classes = 10,
              n_hidden_1 = 128,
              n_hidden_2 = 256,
              learning_rate = 0.1,
              fixed_params = None):

    # Carregando pesos e bias de arquivos NumPy
    if not fixed_params:
        var_names = ['fc1/weights:0', 'fc1/biases:0',
                     'fc2/weights:0', 'fc2/biases:0',
                     'logits/weights:0', 'logits/biases:0',]

        fixed_params = {v: None for v in var_names}
        found_params = False
    else:
        found_params = True

    # Input data
    tf_x = tf.compat.v1.placeholder(tf.float32, [None, n_input], name = 'features')
    tf_y = tf.compat.v1.placeholder(tf.int32, [None], name = 'targets')
    tf_y_onehot = tf.one_hot(tf_y, depth = n_classes, name = 'onehot_targets')

    # Multilayer perceptron
    fc1 = fc_layer(input_tensor = tf_x,
                   n_output_units = n_hidden_1,
                   name = 'fc1',
                   weight_params = fixed_params['fc1/weights:0'],
                   bias_params = fixed_params['fc1/biases:0'],
                   activation_fn = tf.nn.relu)

    fc2 = fc_layer(input_tensor = fc1,
                   n_output_units = n_hidden_2,
                   name = 'fc2',
                   weight_params = fixed_params['fc2/weights:0'],
                   bias_params = fixed_params['fc2/biases:0'],
                   activation_fn = tf.nn.relu)

    logits = fc_layer(input_tensor = fc2,
                      n_output_units = n_classes,
                      name = 'logits',
                      weight_params = fixed_params['logits/weights:0'],
                      bias_params = fixed_params['logits/biases:0'],
                      activation_fn = tf.nn.relu)

    # Loss e optimizer
    ### Somente necessário se nenhum parâmetro existente for encontrado
    ### e um grafo treinável deve ser inicializado
    if not found_params:
        loss = tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=tf.stop_gradient(tf_y_onehot))
        cost = tf.reduce_mean(input_tensor=loss, name='cost')
        optimizer = tf.compat.v1.train.GradientDescentOptimizer(
            learning_rate=learning_rate)
        train = optimizer.minimize(cost, name='train')

    # Previsões
    probabilities = tf.nn.softmax(logits, name = 'probabilities')
    labels = tf.cast(tf.argmax(input=logits, axis=1), tf.int32, name = 'labels')

    correct_prediction = tf.equal(labels, tf_y, name='correct_predictions')
    accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32), name = 'accuracy')

## Treinando e Salvando o Modelo Multilayer Perceptron

In [ ]:
#from tensorflow.examples.tutorials.mnist import input_data
import input_data

##########################
### Configurações
##########################

# Hiperparâmetros
learning_rate = 0.1
training_epochs = 10
batch_size = 64

##########################
### Definição do Grafo
##########################

g = tf.Graph()
with g.as_default():
    mlp_graph()

##########################
### DATASET
##########################

#mnist = input_data.read_data_sets("/media/datasets/DeepLearningI/Cap04/MNIST", one_hot = False)
mnist = input_data.read_data_sets("MNIST", one_hot = False)


##########################
### Treinamento e Avaliação
##########################

with tf.compat.v1.Session(graph = g) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    saver0 = tf.compat.v1.train.Saver()

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = mnist.train.num_examples // batch_size

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run(['train', 'cost:0'], feed_dict = {'features:0': batch_x,
                                                            'targets:0': batch_y})
            avg_cost += c

        train_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.train.images,
                                                        'targets:0': mnist.train.labels})

        valid_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.validation.images,
                                                        'targets:0': mnist.validation.labels})

        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Acurácia em Treino/Validação: %.3f/%.3f" % (train_acc, valid_acc))

    test_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.test.images,
                                                   'targets:0': mnist.test.labels})
    print('Acurácia em Teste: %.3f' % test_acc)

    ##########################
    ### Salvando o Modelo Treinado
    ##########################
    saver0.save(sess, save_path = './mlp')

## Recarregando o Modelo de Arquivos Meta e Checkpoint 

In [ ]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
import input_data

#mnist = input_data.read_data_sets("/media/datasets/DeepLearningI/Cap04/MNIST", one_hot = False)
mnist = input_data.read_data_sets("MNIST", one_hot = False)


with tf.compat.v1.Session() as sess:

    saver1 = tf.compat.v1.train.import_meta_graph('./mlp.meta')
    saver1.restore(sess, save_path = './mlp')

    test_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.test.images,
                                                   'targets:0': mnist.test.labels})
    print('Acurácia em Teste: %.3f' % test_acc)

## Trabalhando com Arquivos NumPy e Criando Grafos Não-Treinados

### Exportando os Parâmetros do Modelo Para Arquivos NumPy NPZ 

In [ ]:
import tensorflow as tf
import numpy as np

tf.compat.v1.reset_default_graph()
with tf.compat.v1.Session() as sess:

    saver1 = tf.compat.v1.train.import_meta_graph('./mlp.meta')
    saver1.restore(sess, save_path='./mlp')

    var_names = [v.name for v in
                 tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES)]

    params = {}
    print('Variáveis encontradas:')
    for v in var_names:
        print(v)

        ary = sess.run(v)
        params[v] = ary

    np.savez('mlp', **params)

### Carregando Arquivos NumPy .npz em `mlp_graph`

In [ ]:
import numpy as np
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
import input_data

###########################
### Carregando Dados e Parâmetros
###########################

#mnist = input_data.read_data_sets("/media/datasets/DeepLearningI/Cap04/MNIST", one_hot = False)
mnist = input_data.read_data_sets("MNIST", one_hot = False)
param_dict = np.load('mlp.npz')

##########################
### Definição do Grafo
##########################


g = tf.Graph()
with g.as_default():

    mlp_graph(fixed_params = param_dict)

with tf.compat.v1.Session(graph=g) as sess:

    test_acc = sess.run('accuracy:0', feed_dict = {'features:0': mnist.test.images,'targets:0': mnist.test.labels})
    print('Acurácia em Teste: %.3f' % test_acc)

# Fim